# प्रॉम्प्ट इंजीनियरिंग का परिचय
प्रॉम्प्ट इंजीनियरिंग वह प्रक्रिया है जिसमें प्राकृतिक भाषा प्रसंस्करण कार्यों के लिए प्रॉम्प्ट्स को डिज़ाइन और बेहतर बनाया जाता है। इसमें सही प्रॉम्प्ट्स का चयन करना, उनके पैरामीटर्स को ट्यून करना, और उनके प्रदर्शन का मूल्यांकन करना शामिल है। प्रॉम्प्ट इंजीनियरिंग एनएलपी मॉडल्स में उच्च सटीकता और दक्षता प्राप्त करने के लिए बहुत महत्वपूर्ण है। इस अनुभाग में, हम OpenAI मॉडल्स का उपयोग करके प्रॉम्प्ट इंजीनियरिंग की मूल बातें जानेंगे।


### अभ्यास 1: टोकनाइज़ेशन
tiktoken का उपयोग करके टोकनाइज़ेशन को एक्सप्लोर करें, जो OpenAI का एक ओपन-सोर्स और तेज़ टोकनाइज़र है  
अधिक उदाहरणों के लिए [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) देखें।


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### अभ्यास 2: Github Models कुंजी सेटअप सत्यापित करें

नीचे दिया गया कोड चलाएँ ताकि आप यह सुनिश्चित कर सकें कि आपका Github Models एंडपॉइंट सही तरीके से सेटअप हुआ है। यह कोड एक साधारण बेसिक प्रॉम्प्ट आज़माता है और उसके कम्प्लीशन को सत्यापित करता है। इनपुट `oh say can you see` का कम्प्लीशन कुछ इस तरह होना चाहिए: `by the dawn's early light..`


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


That line is the opening lyric of "The Star-Spangled Banner," the national anthem of the United States, written by Francis Scott Key. If you'd like more information or analysis, feel free to ask!


### अभ्यास 3: मनगढ़ंत बातें  
यह जानें कि क्या होता है जब आप LLM से ऐसे विषय पर उत्तर देने के लिए कहें, जो शायद अस्तित्व में ही न हो, या ऐसे विषय पर पूछें जिसके बारे में उसे जानकारी न हो क्योंकि वह उसके प्री-ट्रेंड डाटासेट के बाहर (अधिक हालिया) है। देखें कि अगर आप कोई अलग प्रॉम्प्ट या कोई अलग मॉडल आज़माते हैं तो प्रतिक्रिया कैसे बदलती है।


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

### अभ्यास 4: निर्देश आधारित  
"text" वेरिएबल का उपयोग मुख्य सामग्री सेट करने के लिए करें  
और "prompt" वेरिएबल का उपयोग उस मुख्य सामग्री से संबंधित एक निर्देश देने के लिए करें।

यहाँ हम मॉडल से कहते हैं कि वह दिए गए पाठ का सारांश दूसरी कक्षा के छात्र के लिए बनाए।


In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### अभ्यास 5: जटिल प्रॉम्प्ट  
एक ऐसा अनुरोध आज़माएँ जिसमें सिस्टम, यूज़र और असिस्टेंट संदेश हों  
सिस्टम असिस्टेंट का संदर्भ सेट करता है  
यूज़र और असिस्टेंट संदेश बहु-चरणीय बातचीत का संदर्भ प्रदान करते हैं

ध्यान दें कि सिस्टम संदर्भ में असिस्टेंट की पर्सनैलिटी "व्यंग्यात्मक" सेट की गई है।  
कोई अलग पर्सनैलिटी संदर्भ आज़माएँ। या इनपुट/आउटपुट संदेशों की कोई अलग श्रृंखला आज़माएँ


In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### अभ्यास: अपनी सहज बुद्धि को जानें
ऊपर दिए गए उदाहरण आपको ऐसे पैटर्न देते हैं जिनका उपयोग आप नए प्रॉम्प्ट (सरल, जटिल, निर्देश आदि) बनाने के लिए कर सकते हैं - कुछ और अभ्यास बनाने की कोशिश करें ताकि आप उन अन्य विचारों को भी खोज सकें जिनके बारे में हमने बात की है, जैसे उदाहरण, संकेत और बहुत कुछ।



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। यद्यपि हम सटीकता के लिए प्रयासरत हैं, कृपया ध्यान दें कि स्वचालित अनुवादों में त्रुटियाँ या गलतियाँ हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को ही प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
